In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import re, string, unicodedata
import numpy as np
import pandas as pd
import pickle
import collections

import nltk
from nltk import word_tokenize
from nltk import download
from nltk.corpus import stopwords


%pip install stanza
import stanza


     |████████████████████████████████| 342 kB 2.5 MB/s 


In [ ]:
download('stopwords')
download('punkt')
stanza.download('es')
nlp = stanza.Pipeline('es')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


2021-09-06 19:51:29 INFO: Downloading default packages for language: es (Spanish)...


2021-09-06 19:53:25 INFO: Finished downloading models and saved to /root/stanza_resources.
2021-09-06 19:53:25 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |
| ner       | conll02 |

2021-09-06 19:53:25 INFO: Use device: cpu
2021-09-06 19:53:25 INFO: Loading: tokenize
2021-09-06 19:53:25 INFO: Loading: mwt
2021-09-06 19:53:25 INFO: Loading: pos
2021-09-06 19:53:26 INFO: Loading: lemma
2021-09-06 19:53:26 INFO: Loading: depparse
2021-09-06 19:53:27 INFO: Loading: ner
2021-09-06 19:53:30 INFO: Done loading processors!


In [ ]:
def eliminar_textos_basura(texto):
    # eliminar emojis: elimina todo lo que este en <>. 
    texto_procesado = re.sub('<.*?>', ' ', texto)
    # convertir a minusculas
    texto_procesado = texto_procesado.lower()
    # remover @usuario
    texto_procesado = re.sub('@[^\s]+',' ',texto_procesado)
    # remover RT
    texto_procesado = re.sub('rt   ',' ',texto_procesado)
    texto_procesado = re.sub('rt',' ',texto_procesado)
    texto_procesado = re.sub('rt ',' ',texto_procesado)
    # remover numeros
    texto_procesado = re.sub("\d+", "", texto_procesado)
    # texto_procesado = re.sub(r'http\S+', ' ', texto_procesado)
    texto_procesado = re.sub("(\w+:\/\/\S+)", " ", texto_procesado)
    # reemplazar todos los caracteres que no saen alfanuméricos con espacios
    texto_procesado = re.sub(r'[^a-zA-Z0-9ÑñÁáÉéÍíÓóÚú\s]', ' ', texto_procesado)
    
    return texto_procesado

def eliminar_stop_words(texto):
    # obtener lista de stopwords
    stop_words = set(stopwords.words('spanish')) 
    stop_words.remove('no')
    #separar el texto por palabras
    palabras = word_tokenize(texto) 
    # dejar solo palabras que no sean stopwords
    texto_sin_sw = [w for w in palabras if not w in stop_words] 
    #  convertir en una sola cadena la lista de palabras
    texto_sin_sw = ' '.join(texto_sin_sw)

    return texto_sin_sw 

def remove_non_ascii(word):
    """Remueve caracteres no ASCII"""
    new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    
    return new_word

def establecer_lema(text):
  doc = nlp(text)
  new = ''
  for sent in doc.sentences:
    for word in sent.words:
      new = new + word.lemma
      new = new + ' '

  return new

In [ ]:
def ProcesarTweet(Tweet):
  Procesado = eliminar_textos_basura(Tweet)
  Procesado = eliminar_stop_words(Procesado)
  Procesado = remove_non_ascii(Procesado)
  Procesado = establecer_lema(Procesado)
  return Procesado

In [ ]:
def Knn(Matriz, k):
  labels = tipo
  pred_label = []
  etiquetar = []
  indice = 0
  for x in labels:
      if(indice != len(labels)-1):
        if(Matriz[indice]<=0.20): #Filtro que asigan directament 0 cuando el valor de comparacion sera cero o menor a cero
          etiquetar.append((Matriz[indice], 0))
        else:
          etiquetar.append((Matriz[indice], x))
      indice += 1
  etiquetar.sort(reverse=True)
  neighbors = etiquetar[:k]
  votes = []
  for neighbor in neighbors:
            votes.append(neighbor[1])
  counter = collections.Counter(votes) #Determina el que mas se repite
  pred_label.append(counter.most_common()[0][0])
  return pred_label

def EtiquetaEnTexto(valor):
  if valor[0]==1:
    return 'emergencia'
  else:
    return 'no emergencia'

In [ ]:
tipo = [] #Variable que replicara los valores de la columna tipo del datset usado para entrenar los modelos
for i in range(1290):
  tipo.append(1)
for j in range(1380):
  tipo.append(0)

In [ ]:
import pickle

modelLSI = pickle.load(open('/content/gdrive/My Drive/Colab Notebooks/MODELOLSI/Todos5/ModelLSI0.model','rb'))
DiccioLSI = pickle.load(open('/content/gdrive/My Drive/Colab Notebooks/MODELOLSI/Todos5/DiccionarioLSI0.pickle','rb'))
MatrizSimLSI = pickle.load(open('/content/gdrive/My Drive/Colab Notebooks/MODELOLSI/Todos5/MatrizSimilaridadLSI0.pickle','rb'))


In [ ]:
def modeloLSI(tweet):
  Tweet = word_tokenize(ProcesarTweet(tweet)) #Se Preprocesa el tweet y se tokeniza el texto
  
  if len(Tweet) <= 2: #Se valida que el tweet preprocesado tenga un minino de palabras
    Tweet = ['vacio']  #Se le asigna el texto 'vacio' el cual sera detectado como no emergencia
  
  tweet_Dic = DiccioLSI.doc2bow(Tweet) #Se estructura el tweet en función al diccionario del modelo
  sim = MatrizSimLSI[modelLSI[tweet_Dic]] #Se genera un nuevo vector que contiene el nivel de similitud del nuevo tweet con cada tweet del dataset
  
  Valor_Similitud = [] 
  for i in range(len(sim)):
    a = ('%.2f' % sim[i]) #Se redondea cada valor del vector de similitud creado para establecer un mismo formato a todos los valores
    Valor_Similitud.append(float(a))
  
  return Valor_Similitud

In [ ]:
def EtiquetarModelLSI(tweet):
  Similaridad= modeloLSI(tweet)
  k_5 = Knn(Similaridad, 5)
  return k_5 

PRUEBAS EN TIEMPO REAL

Prueba Individual

In [ ]:
def main():
    a = input('Inserte tweet: ')
    re = EtiquetaEnTexto(EtiquetarModelLSI(a))
    return print(re)
main()

Prueba Automatizada con tweets actuales de twitter

In [ ]:
pip install twitter

     |████████████████████████████████| 50 kB 1.8 MB/s 


In [ ]:
import tweepy
import json
import twitter
import time
import datetime
import collections
from urllib.parse import unquote

consumer_key = "uKRmr3hX5coPRt52t4smLu6xh"
consumer_secret = "DkuEhwe2ZFmoGLU0xJhFffe1cqz6rg7TytRYPGF7NhAJBPdl29"
access_token = "2159008194-Em78XPPK4lsqwZpSoYhmlVILubXzrglxaz2kTr6"
access_token_secret = "CjBpk8YKFUlQCvHFmqJI4URWt0t2p19BRdD0Kvl6kDxLS"

auth = auth = twitter.oauth.OAuth(access_token, access_token_secret, consumer_key, consumer_secret)
twitter_api = twitter.Twitter(auth=auth)

In [ ]:
def ConjuntoTweets2(query):
  search_results = twitter_api.search.tweets(q=query, count=100, include_rts= False, type = "recent", lang='es',geocode="-1.83,-78.18,600mi",tweet_mode="extended")
  statuses = search_results['statuses']

# Iterate through 5 more batches of results by following the cursor
  print(search_results['search_metadata'])
  for _ in range(4):
    print("Length of statuses", len(statuses))

    try:
        #print(search_results['search_metadata'])
        next_results = search_results['search_metadata']['next_results']

    except KeyError: # No more results when next_results doesn't exist
        break

    # Create a dictionary from next_results, which has the following form:
    # ?max_id=313519052523986943&q=NCAA&include_entities=1
    kwargs = dict([ kv.split('=') for kv in unquote(next_results[2:]).split("&") ])

    search_results = twitter_api.search.tweets(**kwargs)
    statuses += search_results['statuses']
    print(len(statuses))
  return statuses

In [ ]:
def GuardarTweets(tweets):
  a = []
  b = []
  c = []
  d = []
  for users in tweets: 
    create_at = users['created_at']
    create_at = datetime.datetime.strptime(create_at, '%a %b %d %H:%M:%S %z %Y')   
    try:
      texto = users['full_text']
    except:
      texto = 'none'
    place_name = users['user']['location']
    lista_palabras = place_name.split(',')
    a.append(str(create_at))
    b.append(texto)
    d.append(EtiquetaEnTexto(EtiquetarModelLSI(texto))) 
    try:
      c.append(lista_palabras[0].lower())
    except:
      c.append('none') 
  News_Tweets = pd.DataFrame({'create_at': a, 'text': b, 'place_name':c, 'tipo':d},
                  columns=['create_at', 'text', 'place_name', 'tipo'])
  #print('DatFrame creado con éxito')
  return News_Tweets.drop_duplicates('text')

In [ ]:
palabras = 'peligro OR accidente OR muerte OR muerto OR asesinato OR asesinado OR robo OR ladron OR vehiculo OR transito OR explosion OR incendio OR ayuda'

In [ ]:
#Visualizar los Resultados
Conjunto_tweets = ConjuntoTweets(palabras)
print('Tweets Generados con Exitos')
Data_Tweets = GuardarTweets(Conjunto_tweets)
Data_Tweets = Data_Tweets.reset_index(drop=True)
Data_Tweets

In [ ]:
#Almacenar los resultados
Data_Tweets.to_excel('New_Data_Prueba_62.xlsx', index=False)
!cp New_Data_Prueba_62.xlsx '/content/gdrive/My Drive/Colab Notebooks/Tweets_Etiquetados/'

{'completed_in': 0.157, 'max_id': 1434542861069037568, 'max_id_str': '1434542861069037568', 'next_results': '?max_id=1434537089236979712&q=peligro%20OR%20accidente%20OR%20muerte%20OR%20muerto%20OR%20asesinato%20OR%20asesinado%20OR%20robo%20OR%20ladron%20OR%20vehiculo%20OR%20transito%20OR%20explosion%20OR%20incendio%20OR%20ayuda&geocode=-1.83%2C-78.18%2C600mi&lang=es&count=100&include_entities=1', 'query': 'peligro+OR+accidente+OR+muerte+OR+muerto+OR+asesinato+OR+asesinado+OR+robo+OR+ladron+OR+vehiculo+OR+transito+OR+explosion+OR+incendio+OR+ayuda', 'refresh_url': '?since_id=1434542861069037568&q=peligro%20OR%20accidente%20OR%20muerte%20OR%20muerto%20OR%20asesinato%20OR%20asesinado%20OR%20robo%20OR%20ladron%20OR%20vehiculo%20OR%20transito%20OR%20explosion%20OR%20incendio%20OR%20ayuda&geocode=-1.83%2C-78.18%2C600mi&lang=es&include_entities=1', 'count': 100, 'since_id': 0, 'since_id_str': '0'}
Length of statuses 100
200
Length of statuses 200
300
Length of statuses 300
400
Length of statu

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


Antes de Crear
a =  500  b=  500  c=  500  d=  500
